In [ ]:
!pip3 install vidformer supervision ultralytics

# [Vidformer](https://github.com/ixlab/vidformer): Video Data Transformation

Vidformer uses a `cv2`-compatibility layer allowing `import vidformer.cv2 as cv2` conversion:

In [ ]:
import math

import vidformer as vf
import vidformer.cv2 as cv2
import vidformer.supervision as vf_sv

# Use the api.vidformer.org guest account
# The guest account has few permissions (can't access other videos) and low limits
# To get around this:
#     1) Ask for a regular account
#     2) Use the locally-running YrdenServer instead
#     3) Self-host
server = vf.IgniServer("https://api.vidformer.org", api_key="VF_GUEST")
cv2.set_server(server)

In [ ]:
cap = cv2.VideoCapture(
    "https://vidformer-sample-media.nyc3.digitaloceanspaces.com/tos_720p.mp4"
)
assert cap.isOpened()
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
fps = cap.get(cv2.CAP_PROP_FPS)
out = cv2.VideoWriter(None, cv2.VideoWriter_fourcc(*"mp4v"), 24, (height, width))

# Play the video in the notebook cell (outside jupyter add method="link")
# This will say "Waiting" until you fill in the content in future cells
cv2.vidplay(out)

In [ ]:
radius = 100
center_x, center_y = 300, 300
speed = 2 * math.pi / 100
i = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break
    angle = i * speed
    text_x = int(center_x + radius * math.cos(angle))
    text_y = int(center_y + radius * math.sin(angle))
    cv2.putText(
        frame,
        "Hello, world!",
        (text_x, text_y),
        cv2.FONT_HERSHEY_SIMPLEX,
        1,
        (0, 255, 0),
        2,
        cv2.LINE_AA,
    )
    out.write(frame)
    i += 1
out.release()

## Vidformer for CV Annotation with supervision

In [ ]:
# # Run Yolov8m on the video
# import cv2 as ocv_cv2
# import supervision as sv
# from ultralytics import YOLO

# model = YOLO("yolov8m.pt")

# ocv_cap = ocv_cv2.VideoCapture(
#     "https://vidformer-sample-media.nyc3.digitaloceanspaces.com/tos_720p.mp4"
# )
# assert ocv_cap.isOpened()

# detections = []
# while True:
#   ret, frame = ocv_cap.read()
#   if not ret:
#     break
#   detections.append(sv.Detections.from_ultralytics(model(frame)[0]))


# Load pre-computed detections
import pickle
import urllib

with urllib.request.urlopen(
    "https://vidformer-sample-media.nyc3.cdn.digitaloceanspaces.com/tos_720p-yolov8l-detections.pkl"
) as response:
    detections = pickle.load(response)

out = cv2.VideoWriter(None, cv2.VideoWriter_fourcc(*"mp4v"), 24, (height, width))
cv2.vidplay(out)

In [ ]:
cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

box_anot = vf_sv.BoxAnnotator()
label_anot = vf_sv.LabelAnnotator()
i = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break

    det = detections[i]
    det = det[det.confidence > 0.5]

    labels = [
        f"{class_name} {confidence:.2f}"
        for class_name, confidence in zip(det["class_name"], det.confidence)
    ]
    frame = box_anot.annotate(frame.copy(), det)
    frame = label_anot.annotate(frame.copy(), det, labels)

    out.write(frame)
    i += 1
out.release()